In [1]:
import sklearn
print(sklearn.__version__)

0.20.0


In [2]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from time import time

import numpy as np
import pandas as pd
#from functions.models import 
from functions.datasets import getDatasets
from functions.metrics import evaluator
from functions.plot import ROC, plot_confusion_matrix

In [3]:
baseline = 'baseline1'

In [4]:
dataset_name = 'pan13_en'
task = 'age'

def getBestParams(task, dataset_name):
    
    dataset_name = dataset_name.strip().lower()
    task = task.strip().lower()
    
    # load excel params
    baseline1 = pd.read_excel('./Reports/Reports.xlsx', baseline)
    baseline1['Task'] = baseline1['Task'].str.lower()
    baseline1['Name'] = baseline1['Name'].str.lower()
    
    best_params = baseline1[(baseline1['Name'] == dataset_name) & (baseline1['Task'] == task)]
    
    max_features = best_params['max features'].values[0]
    
    model_params = {
                    'vect__max_features': max_features if max_features != 'None' and not pd.isna(max_features) else None,
                    'vect__max_df': best_params['max df'].values[0] if not pd.isna(best_params['max df'].values[0]) else 1,
                    'clf__C': best_params['C'].values[0] if not pd.isna(best_params['C'].values[0]) else 1000.0, 
                    'clf__penalty': best_params['P'].values[0] if not pd.isna(best_params['P'].values[0]) else 'l2'
                    }
    
    return model_params

getBestParams(task, dataset_name)

{'clf__C': 1000.0,
 'clf__penalty': 'l2',
 'vect__max_df': 1,
 'vect__max_features': None}

In [5]:
def labelEncoder(y):
    le = LabelEncoder()
    le.fit(y)

    return (le.transform(y), len(le.classes_), list(le.classes_))

In [6]:
def clean(text):
    space = ' '    
    for char in ['\n','-', '...', '*', '/', '+', '\\']:
        text = text.replace(char, space)
        
    text = text.replace(space*2, space)
    text = text.replace(space*3, space)
    
    return text.strip()

### Perform classification for each problem / task

In [7]:
def model(X, y, n_classes, classes_name, params):
                
    pipeline = Pipeline([       
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),        
        ('clf', LogisticRegression(verbose=1)),
    ])
    
    # pipeline.set_params(**params)    
    vect = TfidfVectorizer(max_features=params.get('vect__max_features'), max_df=params.get('vect__max_df'))
    
    K = StratifiedKFold(n_splits=10)
    
    t0 = time()
    
    predicted_y = []
    expected_y = []    
    score_y = []
    
    for train_index, test_index in K.split(X, y):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        X_train = vect.fit_transform(X_train)
        X_test = vect.transform(X_test)
        
        clf = LogisticRegression(C=params.get('clf__C'), penalty=params.get('clf__penalty'), solver='liblinear')
        
        clf.fit(X_train, y_train)
        
        predicted_y.extend(clf.predict(X_test))
        expected_y.extend(y_test)
        score_y.extend(clf.predict_proba(X_test))

        ### get train score

    # print("done in %0.2fs and %0.1fmin" % ((time() - t0), ((time() - t0) / 60) ))
    # print()
    
    report = pd.DataFrame(classification_report(expected_y, predicted_y, digits=5, target_names=classes_name, output_dict=True))
    report = report.transpose()
    
    return (
        report, 
        np.asarray(expected_y),
        np.asarray(predicted_y),
        np.asarray(score_y)
        )

In [8]:
def run(task, dataset_name, output = None):    
    datasets = getDatasets(task,'df', dataset_name)
    for i in datasets.iterrows():

        name = i[1]['dataset_name']
        label = task
        ds_path = i[1]['path']

        # load training and test dataframes
        training_path = ds_path + '/' + i[1]['training']        

        df_training = pd.read_csv(training_path)#, usecols=cols)        

        df_training['text'] = df_training['text'].apply(clean)
        X_train = df_training['text'].values
        y_train, n_classes, classes_name = labelEncoder(df_training[label].values)

        # del(df_training)

        # print("Dataset: {0} and task: {1}".format(name, label))

        # print("n_classes: {0}".format(n_classes))

        params = getBestParams(task, dataset_name)
        # print("params: ", params)

        report, expected_y, predicted_y, score_y = model(X_train, y_train, n_classes, classes_name, params)

        # get ROC
        dataset_name = dataset_name + '/' + baseline
        
        roc_c = ROC(expected_y, score_y, n_classes, task, dataset_name, classes_name)
        report['roc'] = list(roc_c.values()) + [roc_c['macro']] * 2

        # compute accuracy
        accuracy = accuracy_score(expected_y, predicted_y)
        report['accuracy'] = [accuracy] * (n_classes + 3)

        # compute confusion matrix
        c_matrix = confusion_matrix(expected_y, predicted_y)
        plot_confusion_matrix(c_matrix, classes_name, task, dataset_name, True)
        cm = pd.DataFrame(c_matrix, columns=classes_name, index=classes_name)

        directory = './Reports/' + task + '/' + dataset_name + '/'
        report.to_csv(directory + 'report.csv')
        cm.to_csv(directory + 'confusion_matrix.csv')    

        print(report)

        # output.put('results for {0} and {1}'.format(task, dataset_name))
        # output.put(report.to_dict())

        pass

In [ ]:
import multiprocessing as mp
import random
import string

random.seed(123)

# Define an output queue
output = mp.Queue()

task_list = ['relig','polit','education','professional','region','TI']#,'gender','age']
dataset_list = ['brmoral','b5post','esic','brblogset','enblogs','pan13_en','pan13_es','sms']

args = []
for task in task_list:
    for ds in dataset_list:
        d = getDatasets(task,'df', ds)
        if len(d.values) > 0:
            args.append([task, ds])

In [ ]:
# Setup a list of processes that we want to run
processes = [mp.Process(target=run, args=(x[0], x[1], output)) for x in args]

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/rafael/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/r

Normalized confusion matrix
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
              f1-score  precision    recall  support  roc  accuracy
r12           0.705570   0.545082  1.000000    133.0  0.5  0.545082
r3            0.000000   0.000000  0.000000     90.0  0.5  0.545082
r45           0.000000   0.000000  0.000000     21.0  0.5  0.545082
micro avg     0.545082   0.545082  0.545082    244.0  0.5  0.545082
macro avg     0.235190   0.181694  0.333333    244.0  0.5  0.545082
weighted avg  0.384594   0.297114  0.545082    244.0  0.5  0.545082
Normalized confusion matrix
[[0.48387097 0.26612903 0.25      ]
 [0.29807692 0.46153846 0.24038462]
 [0.3814433  0.34020619 0.27835052]]Normalized confusion matrix

[[0.30841121 0.69158879]
 [0.1559633  0.8440367 ]]
Normalized confusion matrix
[[0.66071429 0.22321429 0.11607143]
 [0.24793388 0.54545455 0.20661157]
 [0.18478261 0.40217391 0.41304348]]
                                     f1-score  precision    recall  support  \
Básico + Superior incomplet

/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/rafael/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
